In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import warnings
import pandas as pd
from time import sleep
from tqdm import tqdm
warnings.filterwarnings('ignore')


In [2]:
web = webdriver.Chrome(ChromeDriverManager().install())
web.get('https://arenter.ru/catalog/')

[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/yasha_ten/.wdm/drivers/chromedriver/mac64/97.0.4692.71]


In [16]:
def parse_category_main():
    sleep(3)
    global c
    r = 1
    while True:
        web.execute_script("window.scrollTo(0, 1600);")    
        for item in web.find_elements_by_class_name('product-i.optional_product_wrapper.swiper-slide.swiper-slide-active'):
            descr = item.text.split('\n')
            name = descr[0]
            per_day = None
            per_month = None
            for word in descr:
                if 'СУТКИ' in word:
                    per_day = word.replace('c', '₽')
                if 'МЕС' in word:
                    per_month = word.replace('c', '₽')

            data.loc[c] = [sect_name, name, per_day, per_month, r]
            c += 1
            r += 1

        if len(web.find_elements_by_class_name('pagination__nav.pagination__nav--next')) == 0:
            break
        web.find_element_by_class_name('pagination__nav.pagination__nav--next').click() ## ДАЛЬШЕ

        
data = pd.DataFrame({'Категория': [], 'Название': [], 'Аренда День': [], 'Аренда месяц': [], 'Рейтинг': []})
c = 0
base_url = 'https://arenter.ru/catalog/'
sections = web.find_elements_by_css_selector('a.catlist__section-title.link_catalog')
for i in range(len(sections)):
    section = web.find_elements_by_css_selector('a.catlist__section-title.link_catalog')[i]
    sect_name = section.text
    section.click()
    parse_category_main()
    web.get(base_url)




def parse_category():
    sleep(3)
    global c
    while True:
        web.execute_script("window.scrollTo(0, 1600);")   
        for item in web.find_elements_by_class_name('product-i.optional_product_wrapper.swiper-slide.swiper-slide-active'):
            descr = item.text.split('\n')
            name = descr[0]
            per_day = None
            per_month = None
            for word in descr:
                if 'СУТКИ' in word:
                    per_day = word.replace('c', '₽')
                if 'МЕС' in word:
                    per_month = word.replace('c', '₽')
            if name not in data['Название'].values:
                data.loc[c] = [sect_name, name, per_day, per_month, 0]
                c += 1

        if len(web.find_elements_by_class_name('pagination__nav.pagination__nav--next')) == 0:
            break
        web.find_element_by_class_name('pagination__nav.pagination__nav--next').click() ## ДАЛЬШЕ

def wait_until_cond(css):
    flag = 0
    while True:
        try:
            web.find_element_by_class_name(css)
            break
        except Exception:
            if (flag < 10):
                sleep(1)
                flag += 1
            else:
                raise NoSuchElementException


# data = pd.DataFrame({'Категория': [], 'Подкатегория': [], 'Название': [], 'Аренда День': [], 'Аренда месяц': [], 'Рейтинг': []})
base_url = web.current_url
c = len(data)

sections = web.find_elements_by_class_name('catlist__section')

for k in tqdm(range(len(sections))):
    wait_until_cond('catlist__section')
    section = web.find_elements_by_class_name('catlist__section')[k]
    sect_name = section.text.split('\n')[0]
    categories = section.find_elements_by_class_name('catlist__i-title.link_catalog')
    for i in range(len(categories)):
        section = web.find_elements_by_class_name('catlist__section')[k]
        category = section.find_elements_by_class_name('catlist__i-title.link_catalog')[i]
        cat_name = category.text
        category.click()
        parse_category()
        web.get(base_url)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=96.0.4664.55)


In [15]:
data.to_csv('Аренда_01.12.2021.csv')

In [3]:
web.get('https://www.flickr.com/photos/148869261@N02/51718504717/sizes/4k/')

In [8]:
web.find_element_by_class_name('spaceball').get_attribute('href')